# Experimentos sobre el Matrimonio Estable

Este notebook:

1. Genera instancias aleatorias del problema del matrimonio estable (con igual número de hombres y mujeres).  
2. Resuelve cada instancia usando el algoritmo clásico de **Gale–Shapley** de aceptación diferida (con los hombres proponiendo por defecto).  
3. Formula el *poliedro del matrimonio estable* en **PuLP** y resuelve tres programas lineales distintos:  
   * **Regret total** – minimizar la suma de las posiciones (rangos) para ambos lados.  
   * **Regret de los hombres** – minimizar la suma de las posiciones de los proponentes.  
   * **Regret de las mujeres** – minimizar la suma de las posiciones de los receptores.

Al final, puedes comparar los valores objetivo y confirmar que Gale–Shapley (con los hombres proponiendo) es óptimo para los hombres y el peor para las mujeres entre todos los emparejamientos estables.


In [74]:
import random
import itertools
from typing import List, Dict, Tuple
import pulp
from typing import Dict, List, Tuple, Optional
from typing import Dict, List, Tuple

from pulp import (
    LpProblem, LpVariable, LpMinimize, LpStatusOptimal,
    lpSum, listSolvers, PULP_CBC_CMD, GLPK_CMD,LpStatus)


In [75]:

def random_preferences(n: int, seed: Optional[int] = None
                      ) -> Tuple[Dict[int, List[int]], Dict[int, List[int]]]:
    """Return (men_prefs, women_prefs) as dicts mapping index → shuffled list."""
    rng = random.Random(seed)
    men_prefs = {i: rng.sample(range(n), n) for i in range(n)}
    women_prefs = {j: rng.sample(range(n), n) for j in range(n)}
    return men_prefs, women_prefs

In [76]:
def gale_shapley(men_prefs: Dict[int, List[int]],
                 women_prefs: Dict[int, List[int]]
                 ) -> Dict[int, int]:
    n = len(men_prefs)
    free_men = list(men_prefs.keys())
    next_idx = {m: 0 for m in men_prefs}
    engaged_to: Dict[int, int] = {}  # woman→man

    # Build ranking map for women
    women_rank = {w: {m: rank for rank, m in enumerate(pref)}
                  for w, pref in women_prefs.items()}

    while free_men:
        m = free_men.pop(0)
        w = men_prefs[m][next_idx[m]]
        next_idx[m] += 1

        if w not in engaged_to:
            engaged_to[w] = m
        else:
            current = engaged_to[w]
            # Lower rank is better
            if women_rank[w][m] < women_rank[w][current]:
                engaged_to[w] = m
                free_men.append(current)
            else:
                free_men.append(m)

    # return dict man→woman
    return {m: w for w, m in engaged_to.items()}

In [77]:
def ranking(position_list: List[int]) -> Dict[int, int]:
    """Return map item→rank (0 best)."""
    return {x: i for i, x in enumerate(position_list)}

def total_regret(match: Dict[int, int],
                 men_prefs: Dict[int, List[int]],
                 women_prefs: Dict[int, List[int]]
                 ) -> Tuple[int, int, int]:
    men_rank   = {m: ranking(men_prefs[m])[w] for m, w in match.items()}
    women_rank = {w: ranking(women_prefs[w])[m] for m, w in match.items()}
    men_reg   = sum(men_rank.values())
    women_reg = sum(women_rank.values())
    return men_reg + women_reg, men_reg, women_reg

def is_stable(match: Dict[int, int],
              men_prefs: Dict[int, List[int]],
              women_prefs: Dict[int, List[int]]
              ) -> bool:
    n = len(men_prefs)
    women_partner = {v: k for k, v in match.items()}
    women_rank = {w: ranking(pref) for w, pref in women_prefs.items()}

    for m in range(n):
        w_current = match[m]
        men_pref_list = men_prefs[m]
        idx_current = men_pref_list.index(w_current)
        # women that man likes more
        for w in men_pref_list[:idx_current]:
            m_prime = women_partner[w]
            if women_rank[w][m] < women_rank[w][m_prime]:
                return False
    return True

## 1 Formulación del problema de matrimonio estable como un programa lineal

Disponemos de

* un conjunto de hombres $M=\{0,\dots ,n-1\}$,
* un conjunto de mujeres $W=\{0,\dots ,n-1\}$,
* listas de preferencias estrictas `men_prefs[m]` y `women_prefs[w]`.

### 1.1 Variables de decisión

$$
x_{mw} \;=\;
\begin{cases}
1 & \text{si el hombre } m \text{ queda emparejado con la mujer } w,\\[4pt]
0 & \text{en otro caso.}
\end{cases}
$$

Hay $n^{2}$ variables binarias.

### 1.2 Restricciones fundamentales

| Tipo                            | Ecuación                                                                                                                         |
| ------------------------------- | -------------------------------------------------------------------------------------------------------------------------------- |
| **No-negatividad**              | $\displaystyle x_{mw}\ge 0 \quad (\forall\,m,w)$                                                                                 |
| **Asignación (emparejamiento)** | $\displaystyle \sum_{w\in W} x_{mw}=1 \quad (\forall\,m\in M)$<br>$\displaystyle \sum_{m\in M} x_{mw}=1 \quad (\forall\,w\in W)$ |
| **Estabilidad (Vande Vate)**    | para cada $(m,w)$<br>$\displaystyle x_{mw} + \sum_{w'\succ_m w} x_{mw'} + \sum_{m'\succ_w m} x_{m'w} \;\;\ge 1.$                 |

*Las ecuaciones de asignación, junto con $x_{mw}\ge 0$, implican automáticamente $x_{mw}\le 1$; no hace falta añadir esa cota.*

### 1.3 Garantía de integralidad

Con estas tres familias de restricciones se obtiene el **poliedro de matrimonios estables** (Vande Vate 1989; Rothblum 1992), que es **integral**: cada vértice es una matriz 0-1 que describe un emparejamiento estable, y todo emparejamiento estable es un vértice.

---

## 2 Funciones objetivo (medidas de “nostalgia”)

Sea
$\operatorname{rank}_{m}(w)$ la posición (0 = mejor) de $w$ en la lista de $m$, y análogamente $\operatorname{rank}_{w}(m)$.

| Objetivo                     | Expresión lineal                                                                                          | Interpretación                                                               |
| ---------------------------- | --------------------------------------------------------------------------------------------------------- | ---------------------------------------------------------------------------- |
| **Nostalgia total**          | $\displaystyle \min \sum_{m,w} \bigl[\operatorname{rank}_{m}(w)+\operatorname{rank}_{w}(m)\bigr]\;x_{mw}$ | Minimiza la “nostalgia” conjunta de hombres y mujeres.                       |
| **Nostalgia de los hombres** | $\displaystyle \min \sum_{m,w} \operatorname{rank}_{m}(w)\;x_{mw}$                                        | Coincide con la solución óptima del Gale-Shapley donde proponen los hombres. |
| **Nostalgia de las mujeres** | $\displaystyle \min \sum_{m,w} \operatorname{rank}_{w}(m)\;x_{mw}$                                        | Simétrico; lo minimiza Gale-Shapley con propuestas de las mujeres.           |

Como la región factible es integral, basta resolver el PL: el solver devuelve directamente un emparejamiento estable que **optimiza** la medida de nostalgia escogida.


In [57]:
def solve_sm_lp(men_prefs: Dict[int, List[int]],
                women_prefs: Dict[int, List[int]],
                objective: str = "total") -> Dict[int, int]:
    """
    Solve the linear-programming formulation of stable marriage and return a
    matching dict {man: woman}.
    objective ∈ {'total', 'men', 'women'} controls the regret measure.
    """
    n = len(men_prefs)
    prob = pulp.LpProblem("StableMarriage", pulp.LpMinimize)

    # Binary decision variables x[m,w]
    x = {(m, w): pulp.LpVariable(f"x_{m}_{w}", cat="Binary")
         for m in range(n) for w in range(n)}

    # Each man and each woman matched exactly once
    for m in range(n):
        prob += sum(x[m, w] for w in range(n)) == 1
    for w in range(n):
        prob += sum(x[m, w] for m in range(n)) == 1

    # Pre-compute rank maps (0 = best)
    man_rank   = {m: {w: r for r, w in enumerate(pref)}
                  for m, pref in men_prefs.items()}
    woman_rank = {w: {m: r for r, m in enumerate(pref)}
                  for w, pref in women_prefs.items()}

    # No-blocking-pair constraints (NOW WITH x[m,w] INCLUDED!)
    for m in range(n):
        for w in range(n):
            better_women = [w2 for w2 in men_prefs[m]
                            if man_rank[m][w2] < man_rank[m][w]]
            better_men   = [m2 for m2 in women_prefs[w]
                            if woman_rank[w][m2] < woman_rank[w][m]]
            lhs = (x[m, w] +
                   pulp.lpSum(x[m, w2] for w2 in better_women) +
                   pulp.lpSum(x[m2, w] for m2 in better_men))
            prob += lhs >= 1

    # Objective: minimise chosen regret measure
    if objective == "men":
        prob += pulp.lpSum(man_rank[m][w] * x[m, w] for m in range(n) for w in range(n))
    elif objective == "women":
        prob += pulp.lpSum(woman_rank[w][m] * x[m, w] for m in range(n) for w in range(n))
    else:                           # total regret
        prob += pulp.lpSum((man_rank[m][w] + woman_rank[w][m]) * x[m, w]
                           for m in range(n) for w in range(n))

    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    if pulp.LpStatus[prob.status] != "Optimal":
        raise RuntimeError(f"LP status {pulp.LpStatus[prob.status]} – no optimal matching found")

    return {m: w for (m, w), var in x.items() if var.value() > 0.5}

In [73]:
# Demo: generate one instance and solve
n = 80
men_prefs, women_prefs = random_preferences(n, seed=41)

gs_match = gale_shapley(men_prefs, women_prefs)
print("Gale‑Shapley matching:", gs_match)
print("Stable?", is_stable(gs_match, men_prefs, women_prefs))
print("Regrets (total, men, women):", total_regret(gs_match, men_prefs, women_prefs))


for obj in ["total", "men", "women"]:
    lp_match = solve_sm_lp(men_prefs, women_prefs, objective=obj)
    print(f"\nLP optimal for {obj}:", lp_match)
    print("Stable?", is_stable(lp_match, men_prefs, women_prefs))
    print("Regrets:", total_regret(lp_match, men_prefs, women_prefs))

Gale‑Shapley matching: {13: 48, 1: 8, 56: 51, 3: 62, 63: 3, 0: 42, 7: 43, 76: 2, 51: 27, 50: 0, 5: 10, 68: 6, 64: 63, 25: 18, 72: 30, 61: 65, 18: 68, 75: 75, 57: 28, 9: 50, 74: 21, 27: 45, 41: 35, 24: 13, 43: 40, 44: 66, 59: 79, 32: 74, 15: 61, 34: 34, 42: 24, 22: 41, 10: 59, 35: 58, 45: 5, 40: 22, 66: 4, 62: 64, 17: 60, 52: 15, 30: 16, 55: 33, 70: 71, 46: 36, 37: 44, 6: 29, 21: 78, 65: 23, 67: 76, 73: 54, 79: 69, 11: 25, 54: 1, 38: 14, 69: 46, 53: 17, 16: 12, 19: 52, 33: 77, 78: 7, 28: 9, 14: 49, 48: 39, 31: 11, 20: 20, 77: 26, 39: 67, 47: 19, 58: 31, 71: 47, 8: 57, 29: 55, 49: 73, 12: 70, 23: 37, 60: 53, 36: 56, 2: 38, 4: 72, 26: 32}
Stable? True
Regrets (total, men, women): (1799, 284, 1515)

LP optimal for total: {0: 42, 1: 24, 2: 1, 3: 11, 4: 73, 5: 9, 6: 29, 7: 43, 8: 57, 9: 56, 10: 59, 11: 25, 12: 39, 13: 34, 14: 54, 15: 61, 16: 12, 17: 8, 18: 63, 19: 38, 20: 20, 21: 48, 22: 41, 23: 37, 24: 15, 25: 65, 26: 72, 27: 45, 28: 58, 29: 36, 30: 16, 31: 19, 32: 74, 33: 55, 34: 10, 35: 1